## Playground season 3, episode 07
### Start: February 14, 2023
### End: February 27, 2023

## EDA adoptation
---
Some of the EDA methods and convert took from: https://www.kaggle.com/code/jcaliz/ps-s03e07-a-complete-eda

In [1]:
import pandas as pd
import numpy as np
import os

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns

import math
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.ticker import MaxNLocator


In [2]:
if os.name == 'nt':
    data_path = r"D:\Coding_pratice\_Data\kaggle_playground_series\playground-series-s3e7"
else:
    data_path = '/Users/admin/_Work/Data/Practice/playground-series-s3e7'
    
train_csv = os.path.join(data_path, 'train.csv')
test_csv = os.path.join(data_path, 'test.csv')
hotel_csv = os.path.join(data_path, 'hotel_bookings.csv')
train_reservation_csv = os.path.join(data_path, 'Reservation Cancellation Prediction/train__dataset.csv')
test_reservation_csv = os.path.join(data_path, 'Reservation Cancellation Prediction/test___dataset.csv')
hotel_res_csv = os.path.join(data_path, 'Hotel Reservations.csv')

train_df = pd.read_csv(train_csv, index_col=0)
test_df = pd.read_csv(test_csv, index_col=0)
origin_train_df = pd.read_csv(train_reservation_csv)
origin_test_df = pd.read_csv(test_reservation_csv)
hotel_demand_df = pd.read_csv(hotel_csv)
hotel_res_df = pd.read_csv(hotel_res_csv, index_col=0)

"hotel_booking.csv" - Source: https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand

"Reservation Cancellation Prediction/xxx_dataset.csv" - Source: https://www.kaggle.com/datasets/gauravduttakiit/reservation-cancellation-prediction

"Hotel Reservations.csv" - Source: https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset

In [3]:
hotel_res_df.index.name = 'id'

## Feature notes
---
* *id*: unique identifier of each booking

* *no_of_adults*: Number of adults
* *no_of_children*: Number of children
* *no_of_weekend_nights*: Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel
* *no_of_week_nights*: Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel
* *type_of_meal_plan*: Type of meal plan booked by the customer
* *required_car_parking_space*: Does the customer require a car parking space? (0 - No, 1 - Yes)
* *room_type_reserved*: Type of room reserved by the customer. The values are ciphered (encoded) by INN Hotels.
* *lead_time*: Number of days between the date of booking and the arrival date
* *arrival_year*: Year of arrival date
* *arrival_month*: Month of arrival date
* *arrival_date*: Date of the month
* *market_segment_type*: Market segment designation
* *repeated_guest*: Is the customer a repeated guest? (0 - No, 1 - Yes)
* *no_of_previous_cancellations*: Number of previous bookings that were canceled by the customer prior to the current booking
* *no_of_previous_bookings_not_canceled*: Number of previous bookings not canceled by the customer prior to the current booking
* *avg_price_per_room*: Average price per day of the reservation; prices of the rooms are dynamic. (in euros)
* *no_of_special_requests*: Total number of special requests made by the customer (e.g. high floor, view from the room, etc)
* *booking_status*: Flag indicating if the booking was canceled or not.

Rename 'hotel_demand_df' columns to match train_df

In [4]:
hotel_demand_df.rename(
    columns= {
    'adults': 'no_of_adults',
    'children': 'no_of_children',
    'stays_in_weekend_nights': 'no_of_weekend_nights',
    'stays_in_week_nights': 'no_of_week_nights',
    'meal': 'type_of_meal_plan',
    'required_car_parking_spaces': 'required_car_parking_space',
    'reserved_room_type': 'room_type_reserved',
    'lead_time': 'lead_time',
    'arrival_date_year': 'arrival_year',
    'arrival_date_month': 'arrival_month',
    'arrival_date_day_of_month': 'arrival_date',
    'market_segment': 'market_segment_type',
    'is_repeated_guest': 'repeated_guest',
    'previous_cancellations': 'no_of_previous_cancellations',
    'previous_bookings_not_canceled': 'no_of_previous_bookings_not_canceled',
    'adr': 'avg_price_per_room',
    'total_of_special_requests': 'no_of_special_requests',
    'is_canceled': 'booking_status'
}, inplace=True)

In [5]:
hotel_demand_df['arrival_month'] = hotel_demand_df['arrival_month'].map({
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
})

hotel_demand_df['type_of_meal_plan'] = hotel_demand_df['type_of_meal_plan'].map({
        'BB': 0, 'HB': 2, 'SC': 1, 'Undefined': 1, 'FB': 3
})

hotel_demand_df['market_segment_type'] = hotel_demand_df['market_segment_type'].map({
        "Online TA": 1, "Offline TA/TO": 0, "Corporate": 2, "Complementary": 4, "Aviation": 3
})

hotel_demand_df['room_type_reserved'] = hotel_demand_df['room_type_reserved'].map({
        'A':  0, 'D':  1, 'E':  3, 'F':  2, 'G':  4, 'B':  5, 'C':  6
})

hotel_demand_df['market_segment_type'].fillna(5, inplace=True)
hotel_demand_df['room_type_reserved'].fillna(7, inplace=True)

hotel_res_df['type_of_meal_plan'] = hotel_res_df['type_of_meal_plan'].map(
    {
        'Meal Plan 1': 0, 'Not Selected': 2, 'Meal Plan 2': 1, 'Meal Plan 3': 3
    }
)
hotel_res_df['type_of_meal_plan'] = hotel_res_df['type_of_meal_plan'].map(
    {
        'Room_Type 1': 0, 'Room_Type 4': 1, 'Room_Type 6': 3, 
        'Room_Type 2': 2, 'Room_Type 5': 4, 'Room_Type 7': 5, 'Room_Type 3': 6
    }
)
hotel_res_df['market_segment_type'] = hotel_res_df['market_segment_type'].map(
    {
        "Online": 1, "Offline": 0, "Corporate": 2, "Complementary": 4, "Aviation": 3
    }
)
hotel_res_df['booking_status'] = hotel_res_df['booking_status'].map(
    {
        "Not_Canceled": 0, "Canceled": 1,
    }
)

In [6]:
print("Train shape:         ", train_df.shape)
print("Test shape:          ", test_df.shape)
print("Origin Train shape:  ", origin_train_df.shape)
print("Origin Test shape:   ", origin_test_df.shape)
print("Hotel demand shape:  ", hotel_demand_df.shape)
print("Hotel reserve shape: ", hotel_res_df.shape)

Train shape:          (42100, 18)
Test shape:           (28068, 17)
Origin Train shape:   (18137, 18)
Origin Test shape:    (18138, 17)
Hotel demand shape:   (119390, 32)
Hotel reserve shape:  (36275, 18)


### Distribution

Check numerical + ordinal featues using plotly

In [16]:
from functools import reduce
# Merge all dataframe
total_df = pd.concat([
    train_df.assign(set='train'),
    test_df.assign(set='test'),
    origin_train_df.assign(set='origin_train'),
    origin_test_df.assign(set='origin_test'),
    hotel_demand_df.assign(set='demand')
], ignore_index=True)

total_df.reset_index(inplace=True, drop=True)

def add_ordinal(fig, feature, position=(1,1)):
    percentage = total_df.groupby('set')[feature].value_counts(True)
    percentage = percentage.rename('%').reset_index()
    dataframe = percentage.set.unique()
    data_frames = [percentage[percentage.set==i] for i in dataframe]
    display(data_frames[0].set[0])
    df_merge = reduce(lambda left, right: pd.merge(
        left, right, on=feature, how='outer', suffixes=('_', '_')
        ), data_frames).fillna(0)
    display(df_merge)
    # for i, name in enumerate(percentage['set'].unique()):
    #     fig.add_trace(
    #         go.Bar(
    #             x= percentage[percentage['set'] == name][feature],
    #             y= percentage[percentage['set'] == name]['%'],
    #             name = name,
    #             offsetgroup = i + 1
    #         ), row=1, col=1
    #     )
    
def add_numeric():
    pass

labels = ['train', 'test', 'origin_train', 'origin_test', 'demand']

ordinal_features = [
    'no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights',
    'no_of_special_requests', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled'
]

numeric_features = [
    'lead_time', 'avg_price_per_room'
]
# columns =  ordinal_features + numeric_features
columns = ['no_of_adults']
fig = make_subplots(
    rows= 1,
    cols= 1,
    subplot_titles=['Testing']

)
for i, column in enumerate(columns):
    if column in ordinal_features:
        add_ordinal(fig, column, (i//2 + 1, i%2 + 1))
    else:
        add_numeric()


'demand'

/var/folders/2z/c11mbd_x6dn12d_ykf68p7gr0000gn/T/ipykernel_5750/3984382100.py:19: FutureWarning:

Passing 'suffixes' which cause duplicate columns {'%_', 'set_'} in the result is deprecated and will raise a MergeError in a future version.



,set_,no_of_adults,%_,set_,%_,set_,%_,set_,%_,set,%
0,demand,2,0.751152,origin_test,0.716948,origin_train,0.722501,test,0.729229,train,0.730903
1,demand,1,0.192872,origin_test,0.214246,origin_train,0.210013,test,0.171548,train,0.168385
2,demand,3,0.051947,origin_test,0.064340,origin_train,0.063406,test,0.094770,train,0.096461
3,demand,0,0.003375,origin_test,0.003970,origin_train,0.003694,test,0.004204,train,0.003967
4,demand,4,0.000519,origin_test,0.000496,origin_train,0.000386,test,0.000249,train,0.000285
5,demand,26,0.000042,0,0.000000,0,0.000000,0,0.000000,0,0.000000
6,demand,5,0.000017,0,0.000000,0,0.000000,0,0.000000,0,0.000000
7,demand,20,0.000017,0,0.000000,0,0.000000,0,0.000000,0,0.000000
8,demand,27,0.000017,0,0.000000,0,0.000000,0,0.000000,0,0.000000
9,demand,6,0.000008,0,0.000000,0,0.000000,0,0.000000,0,0.000000
